In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats 

sns.set_theme()

In [2]:
import sys
import os


module_path =os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\scripts")
    
    


In [3]:
from univariate import Univariate,readCSV
from loaddata import ReadData
from aggregarefunctions import aggregateClass

In [4]:
dataRead=ReadData('../data/Processed_data.csv')
data=dataRead.readCsv()

Reading Data...

Successfully read the dataset ...


In [5]:
data.head()

,Dur. (ms),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),...,Year,Hour,Avg Bearer TP DL (Mbps),Avg Bearer TP UL (Mbps),Activity Duration DL (s),Activity Duration UL (s),Total Session (Mbs),five Class,Decile Class,Decile Class Category
0,1823652.0,42.000000,5.000000,23.0,44.0,100.0,0.0,0.0,0.0,100.0,...,2019,12,0.022461,0.042969,37.624,38.787,493.406131,Low Usage,1,Outlier
1,1365104.0,65.000000,5.000000,16.0,26.0,100.0,0.0,0.0,0.0,100.0,...,2019,13,0.015625,0.025391,0.168,3.560,1176.919550,Outlier,8,Moderately High Usage
2,1361762.0,109.795706,17.662883,6.0,9.0,100.0,0.0,0.0,0.0,100.0,...,2019,17,0.005859,0.008789,0.000,0.000,685.103951,Moderate Usage,2,Extremely Low
3,1321509.0,109.795706,17.662883,44.0,44.0,100.0,0.0,0.0,0.0,100.0,...,2019,0,0.042969,0.042969,3.330,37.882,1562.493020,Outlier,9,Extreme High Usage
4,1089009.0,109.795706,17.662883,6.0,9.0,100.0,0.0,0.0,0.0,100.0,...,2019,20,0.005859,0.008789,0.000,0.000,1104.727653,Extreme Usage,7,High Usage


In [6]:
aggFunc=aggregateClass(data)


In [7]:
aggFunc.aggregateSum('Year','Total Session (Mbs)','five Class','Dur. (ms).1',5)

,Total Session (Mbs),five Class,Dur. (ms).1
0,1.311595e+08,Low UsageOutlierModerate UsageOutlierExtreme U...,1.569139e+08


In [8]:
aggFunc.aggregateCategorical('Hour','Total Session (Mbs)','five Class',3)

TypeError: aggregateCategorical() missing 1 required positional argument: 'top'